In [1]:
import re
from datetime import datetime
import pandas as pd


In [2]:
df = pd.read_excel('../data/ModulesCombinationv5.xlsx', 'Sheet3')

In [3]:
df

,Unit,Class,Lecture,Tutorial
0,ADM1000,Class 1,Mon 4.30pm to 6pm,Tue 3pm to 5.30pm
1,ADM1000,Class 2,Mon 4.30pm to 6pm,Wed 1.45pm to 4.15pm
2,ADM1000,Class 3,Mon 4.30pm to 6pm,Wed 4.15pm to 6.45pm
3,ADM1000,Class 4,Wed 9.30am to 11am,Thu 9am to 11.30am
4,ADM1000,Class 5,Wed 9.30am to 11am,Thu 12.30pm to 3pm
5,ADM1000,Class 6,Wed 9.30am to 11am,Thu 3pm to 5.30pm
6,ADM1000,Class 7,Wed 11.55am to 12.45pm,Fri 9am to 11.30am
7,ADM1000,Class 8,Wed 11.55am to 12.45pm,Fri 12.30pm to 3pm
8,ADM1000,Class 9,Wed 11.55am to 12.45pm,Fri 3pm to 5.30pm
9,HRM2008,Class 1,Mon 9am to 10.30am,Mon 1.15pm to 3.45pm


In [ ]:
df.columns

In [5]:
for k, v in df['Unit'].value_counts().items():
    print(k)

MAB2002
ADM1000
HRM2008
IBIS1100


In [ ]:
df = df[['Unit', 'Class', 'Lecture', 'Tutorial']]

In [ ]:
unit_map = {
    'MAB2002': 'Module A',
    'ADM1000': 'Module B',
    'HRM2008': 'Module C',
    'IBIS1100': 'Module D'
}

In [ ]:
unit_map = {
    'ADM1000': 'Module A',
    'ACL2012': 'Module B',
    'INFP2000': 'Module C',
    'IECO1100': 'Module D'
}

In [ ]:
unit_map = {
    'MAB2002': 'Module A',
    'ADM1000': 'Module B',
    'HRM2008': 'Module C',
    'IBIS1100': 'Module D'
}

In [ ]:
# Fix the parsing issue by ensuring all times are extracted accurately
def refine_time_parsing(df, time_column):
    """
    Improves the parsing of time ranges by handling edge cases and filling in missing values.
    """

    def time_to_24h(t):
        """Convert 12-hour time format to 24-hour time."""
        return datetime.strptime(t, '%I%p').time() if '.' not in t else datetime.strptime(t, '%I.%M%p').time()

    starts, ends = [], []
    for entry in df[time_column]:
        match = re.search(r'(\d+\.?\d*\w+)\s*to\s*(\d+\.?\d*\w+)', str(entry))
        if match:
            starts.append(time_to_24h(match.group(1)))
            ends.append(time_to_24h(match.group(2)))
        else:
            starts.append(None)
            ends.append(None)

    df[f'{time_column}_Start'] = starts
    df[f'{time_column}_End'] = ends
    return df

# Reapply refined parsing to lecture and tutorial times
df_cleaned = refine_time_parsing(df, 'Lecture')
df_cleaned = refine_time_parsing(df, 'Tutorial')

# Verify the refined times are parsed correctly
df_cleaned.head()

In [ ]:
def check_overlap(first_class, second_class):
    '''
    class: (day, start_time, end_time)
    '''
    if first_class[1] == None:
        return False
    
    return first_class[1] < second_class[2] and second_class[1] < first_class[2] and first_class[0] == second_class[0]

In [ ]:
df_cleaned['lecture_day'] = df['Lecture'].apply(lambda x: x.split(' ')[0])
df_cleaned['tutorial_day'] = df['Tutorial'].apply(lambda x: x.split(' ')[0])

In [ ]:
df_cleaned

In [ ]:
def check_conflict(current_schedule, new_class):
    for s in current_schedule:
        # check lecture with lecture
        class_1 = (s['lecture_day'], s['Lecture_Start'], s['Lecture_End'])
        class_2 = (new_class['lecture_day'], new_class['Lecture_Start'], new_class['Lecture_End'])
        if check_overlap(class_1, class_2):
#             print('Lecture conflict found!')
#             print('class 1: ', class_1)
#             print('class 2: ', class_2)
            return True
        
        # check lecture with tutorial
        class_1 = (s['lecture_day'], s['Lecture_Start'], s['Lecture_End'])
        class_2 = (new_class['tutorial_day'], new_class['Tutorial_Start'], new_class['Tutorial_End'])
        if check_overlap(class_1, class_2):
#             print('Lecture conflict found!')
#             print('class 1: ', class_1)
#             print('class 2: ', class_2)
            return True

        # check tutorial with lecture
        class_1 = (s['tutorial_day'], s['Tutorial_Start'], s['Tutorial_End'])
        class_2 = (new_class['lecture_day'], new_class['Lecture_Start'], new_class['Lecture_End'])
        if check_overlap(class_1, class_2):
#             print('Tutorial conflict found!')
#             print('class 1: ', class_1)
#             print('class 2: ', class_2)

            return True
        # check tutorial with tutorial
        class_1 = (s['tutorial_day'], s['Tutorial_Start'], s['Tutorial_End'])
        class_2 = (new_class['tutorial_day'], new_class['Tutorial_Start'], new_class['Tutorial_End'])
        if check_overlap(class_1, class_2):
#             print('Tutorial conflict found!')
#             print('class 1: ', class_1)
#             print('class 2: ', class_2)

            return True
    return False
        

In [ ]:
df_cleaned['unit_mapped'] = df_cleaned['Unit'].apply(lambda x: unit_map[x])

In [ ]:
df_cleaned

In [ ]:
A_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module A']
B_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module B']
C_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module C']
D_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module D']

In [ ]:
solution = []
while len(solution) < 3:
    current_schedule = []
    for i_a, row_a in A_df.iterrows():
        if check_conflict(current_schedule, row_a):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_a)
            A_df.drop(i_a, inplace=True)
            break

    for i_b, row_b in B_df.iterrows():
        if check_conflict(current_schedule, row_b):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_b)
            B_df.drop(i_b, inplace=True)
            break

    for i_c, row_c in C_df.iterrows():
        if check_conflict(current_schedule, row_c):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_c)
            C_df.drop(i_c, inplace=True)
            break

    for i_d, row_d in D_df.iterrows():
        if check_conflict(current_schedule, row_d):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_d)
            D_df.drop(i_d, inplace=True)
            break

    solution.append(current_schedule)

In [ ]:
for s in solution:
    selected_days = []
    for c in s:
        print(f"{c['Unit']} | {c['Class']} | {c['Lecture']} | {c['Tutorial']}")
        selected_days.append(c['lecture_day'])
        selected_days.append(c['tutorial_day'])

    selected_days = list(set(selected_days))
    print('selected days: ', selected_days)
    print('-'*88)

# Caring about the days

In [ ]:
A_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module A']
B_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module B']
C_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module C']
D_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module D']

In [ ]:
solution = []
while len(solution) < 3:
    current_schedule = []
    selected_days = []
    print(len(solution), '*'*30)
    for i_a, row_a in A_df.iterrows():
        if check_conflict(current_schedule, row_a):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_a)
            A_df.drop(i_a, inplace=True)
            selected_days.append(row_a['lecture_day'])
            selected_days.append(row_a['tutorial_day'])
            
            selected_days = list(set(selected_days))
            print('selected days: ', selected_days)
            break

    B_filtered_df = B_df[(~B_df['lecture_day'].isin(selected_days)) | (~B_df['tutorial_day'].isin(selected_days)) ]
    B_continue = False
    for i_b, row_b in B_filtered_df.iterrows():
        if check_conflict(current_schedule, row_b):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_b)
            B_df.drop(i_b, inplace=True)
            B_continue = True
            selected_days.append(row_b['lecture_day'])
            selected_days.append(row_b['tutorial_day'])
            
            selected_days = list(set(selected_days))
            print('selected days: ', selected_days)

            
            break
    if not B_continue:
        for i_b, row_b in B_df.iterrows():
            if check_conflict(current_schedule, row_b):
                # there is conflict
                continue
            else:
                # append to current_schedule and remove from selection
                current_schedule.append(row_b)
                B_df.drop(i_b, inplace=True)
                selected_days.append(row_b['lecture_day'])
                selected_days.append(row_b['tutorial_day'])

                selected_days = list(set(selected_days))
                print('selected days: ', selected_days)
                break
    
    C_filtered_df = C_df[(~C_df['lecture_day'].isin(selected_days)) | (~C_df['tutorial_day'].isin(selected_days)) ]
    C_continue = False
    for i_c, row_c in C_filtered_df.iterrows():
        if check_conflict(current_schedule, row_c):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_c)
            C_df.drop(i_c, inplace=True)
            C_continue = True
            selected_days.append(row_c['lecture_day'])
            selected_days.append(row_c['tutorial_day'])
            
            selected_days = list(set(selected_days))
            print('selected days: ', selected_days)
            break
    if not C_continue:
        for i_c, row_c in C_df.iterrows():
            if check_conflict(current_schedule, row_c):
                # there is conflict
                continue
            else:
                # append to current_schedule and remove from selection
                current_schedule.append(row_c)
                C_df.drop(i_c, inplace=True)
                selected_days.append(row_c['lecture_day'])
                selected_days.append(row_c['tutorial_day'])

                selected_days = list(set(selected_days))
                print('selected days: ', selected_days)
                break

    for i_d, row_d in D_df.iterrows():
        if check_conflict(current_schedule, row_d):
            # there is conflict
            continue
        else:
            # append to current_schedule and remove from selection
            current_schedule.append(row_d)
            D_df.drop(i_d, inplace=True)
            break

    solution.append(current_schedule)

In [ ]:
for s in solution:
    selected_days = []
    for c in s:
        print(f"{c['Unit']} | {c['Class']} | {c['Lecture']} | {c['Tutorial']}")
        selected_days.append(c['lecture_day'])
        selected_days.append(c['tutorial_day'])

    selected_days = list(set(selected_days))
#     print('selected days: ', selected_days)
    print('-'*88)

# Shuffling

In [ ]:
df_cleaned

In [ ]:
def check_solution_is_valid(solution, num_combi, num_units):
    if len(solution) != num_combi:
        return False
    for s in solution:
        if len(s) != num_units:
            return False
        
    return True

In [ ]:
num_combi = 7
num_units = 4

In [ ]:
solution = []

while not check_solution_is_valid(solution, num_combi, num_units):
    print('finding solution...')
    A_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module A'].sample(frac=1)
    B_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module B'].sample(frac=1)
    C_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module C'].sample(frac=1)
    D_df = df_cleaned[df_cleaned['unit_mapped'] == 'Module D'].sample(frac=1)
    solution = []
    while len(solution) < num_combi:
        current_schedule = []
        selected_days = []
        for i_a, row_a in A_df.iterrows():
            if check_conflict(current_schedule, row_a):
                # there is conflict
                continue
            else:
                # append to current_schedule and remove from selection
                current_schedule.append(row_a)
                A_df.drop(i_a, inplace=True)
                selected_days.append(row_a['lecture_day'])
                selected_days.append(row_a['tutorial_day'])

                selected_days = list(set(selected_days))
#                 print('selected days: ', selected_days)
                break

        B_filtered_df = B_df[(~B_df['lecture_day'].isin(selected_days)) | (~B_df['tutorial_day'].isin(selected_days)) ]
        B_continue = False
        for i_b, row_b in B_filtered_df.iterrows():
            if check_conflict(current_schedule, row_b):
                # there is conflict
                continue
            else:
                # append to current_schedule and remove from selection
                current_schedule.append(row_b)
                B_df.drop(i_b, inplace=True)
                B_continue = True
                selected_days.append(row_b['lecture_day'])
                selected_days.append(row_b['tutorial_day'])

                selected_days = list(set(selected_days))
#                 print('selected days: ', selected_days)


                break
        if not B_continue:
            for i_b, row_b in B_df.iterrows():
                if check_conflict(current_schedule, row_b):
                    # there is conflict
                    continue
                else:
                    # append to current_schedule and remove from selection
                    current_schedule.append(row_b)
                    B_df.drop(i_b, inplace=True)
                    selected_days.append(row_b['lecture_day'])
                    selected_days.append(row_b['tutorial_day'])

                    selected_days = list(set(selected_days))
#                     print('selected days: ', selected_days)
                    break

        C_filtered_df = C_df[(~C_df['lecture_day'].isin(selected_days)) | (~C_df['tutorial_day'].isin(selected_days)) ]
        C_continue = False
        for i_c, row_c in C_filtered_df.iterrows():
            if check_conflict(current_schedule, row_c):
                # there is conflict
                continue
            else:
                # append to current_schedule and remove from selection
                current_schedule.append(row_c)
                C_df.drop(i_c, inplace=True)
                C_continue = True
                selected_days.append(row_c['lecture_day'])
                selected_days.append(row_c['tutorial_day'])

                selected_days = list(set(selected_days))
#                 print('selected days: ', selected_days)
                break
        if not C_continue:
            for i_c, row_c in C_df.iterrows():
                if check_conflict(current_schedule, row_c):
                    # there is conflict
                    continue
                else:
                    # append to current_schedule and remove from selection
                    current_schedule.append(row_c)
                    C_df.drop(i_c, inplace=True)
                    selected_days.append(row_c['lecture_day'])
                    selected_days.append(row_c['tutorial_day'])

                    selected_days = list(set(selected_days))
#                     print('selected days: ', selected_days)
                    break

        for i_d, row_d in D_df.iterrows():
            if check_conflict(current_schedule, row_d):
                # there is conflict
                continue
            else:
                # append to current_schedule and remove from selection
                current_schedule.append(row_d)
                D_df.drop(i_d, inplace=True)
                break

        solution.append(current_schedule)
print('solution found!')

In [ ]:
for s in solution:
    selected_days = []
    for c in s:
        print(f"{c['Unit']} | {c['Class']} | {c['Lecture']} | {c['Tutorial']}")
        selected_days.append(c['lecture_day'])
        selected_days.append(c['tutorial_day'])

    selected_days = list(set(selected_days))
#     print('selected days: ', selected_days)
    print('-'*88)